In [ ]:
import torch.nn as nn
import torchvision
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
import torch
from torch.utils.data.dataloader import DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

import matplotlib.pyplot as plt 
import numpy as np
import sys

In [ ]:
# import model from the kuangliu repo 
from resnet import ResNet18

In [ ]:
from torchvision.transforms.v2 import MixUp

mixup_transform = MixUp(num_classes=10)

In [ ]:
# hyperparameters
batch_size = 64
num_epochs = 30
learning_rate = 0.001
is_scheduler = True
weight = 0.0005
is_mixup = False

In [ ]:
## Normalization adapted for CIFAR10
normalize_scratch = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

# Transforms is a list of transformations applied on the 'raw' dataset before the data is fed to the network.
# Here, Data augmentation (RandomCrop and Horizontal Flip) are applied to each batch, differently at each epoch, on the training set data only
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize_scratch,
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    normalize_scratch,
])

### The data from CIFAR10 are already downloaded in the following folder
rootdir = '/opt/img/effdl-cifar10/'

c10train = CIFAR10(rootdir,train=True,download=True,transform=transform_train)
c10test = CIFAR10(rootdir,train=False,download=True,transform=transform_test)

trainloader = DataLoader(c10train,batch_size=batch_size,shuffle=True)
testloader = DataLoader(c10test,batch_size=batch_size)

In [ ]:
## number of target samples for the final dataset
num_train_examples = len(c10train)
num_samples_subset = 15000

## We set a seed manually so as to reproduce the results easily
seed  = 2147483647

## Generate a list of shuffled indices ; with the fixed seed, the permutation will always be the same, for reproducibility
indices = list(range(num_train_examples))
np.random.RandomState(seed=seed).shuffle(indices)## modifies the list in place

## We define the Subset using the generated indices
c10train_subset = torch.utils.data.Subset(c10train,indices[:num_samples_subset])
print(f"Initial CIFAR10 dataset has {len(c10train)} samples")
print(f"Subset of CIFAR10 dataset has {len(c10train_subset)} samples")

# Finally we can define anoter dataloader for the training data
trainloader_subset = DataLoader(c10train_subset,batch_size=batch_size,shuffle=True)
### You can now use either trainloader (full CIFAR10) or trainloader_subset (subset of CIFAR10) to train your networks.

## TASK 1. Train a model from scratch

In [ ]:
# device conf : using GPU if it is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# define the model used
model = ResNet18()
model = model.to(device)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight)
#optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Define a schedule which gonna divide the lr by 10 when reaching a plateau in performance for the accuracy
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1, verbose=True)

In [ ]:
# use a scheduler to change the learning rate
# import librairies

In [ ]:
def train_model(model, trainloader, testloader, epochs, is_scheduler):
    train_accuracy_list, test_accuracy_list = [], []
    train_loss_list, test_loss_list = [], []
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # Training accuracy
            _, predicted = outputs.max(1)
            correct_train += predicted.eq(labels).sum().item()
            total_train += labels.size(0)

        train_accuracy = 100 * correct_train / total_train
        train_accuracy_list.append(train_accuracy)
        train_loss_list.append(running_loss / len(trainloader))

        # Evaluate on test set
        model.eval()
        correct_test = 0
        total_test = 0
        test_loss = 0.0

        with torch.no_grad():
            for images, labels in testloader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                test_loss += loss.item()

                _, predicted = outputs.max(1)
                correct_test += predicted.eq(labels).sum().item()
                total_test += labels.size(0)

        test_accuracy = 100 * (correct_test / total_test)
        test_accuracy_list.append(test_accuracy)
        test_loss_list.append(test_loss / len(testloader))

        # if using a scheduler
        current_lr = optimizer.param_groups[0]['lr']
        if is_scheduler:
            scheduler.step(test_loss / len(testloader))
            if current_lr != optimizer.param_groups[0]['lr']:
                print(f"LEARNING RATE UPDATE TO:{optimizer.param_groups[0]['lr']}")

        print(f"Epoch {epoch+1}: "
              f"Train Loss: {running_loss/len(trainloader):.4f}, "
              f"Train Acc: {train_accuracy:.2f}%, "
              f"Test Acc: {test_accuracy:.2f}% ",
              f"Learning Rate: {optimizer.param_groups[0]['lr']}")

    print("Training complete my boss")

    ############# PLOT THE LOSS AND THE ACCURACY PER EPOCH ####################
    epochs = range(1, num_epochs+1)

    fig, axes = plt.subplots(1, 2, figsize=(10, 4)) 
    
    # Loss plot
    axes[0].plot(epochs, train_loss_list, label="Training loss", color="blue")
    axes[0].plot(epochs, test_loss_list, label="Testing loss", color="red")
    axes[0].set_title("Training and testing loss")
    axes[0].set_xlabel("Epochs")
    axes[0].set_ylabel("Loss")
    axes[0].legend()
    
    # Accuracy plot
    axes[1].plot(epochs, train_accuracy_list, label="Training accuracy", color="blue")
    axes[1].plot(epochs, test_accuracy_list, label="Testing accuracy", color="red")
    axes[1].set_title("Training and Testing accuracy")
    axes[1].set_xlabel("Epochs")
    axes[1].set_ylabel("Accuracy (%)")
    axes[1].legend()
    
    plt.tight_layout() 
    plt.show()
    #return train_loss_list, test_loss_list, train_accuracy_list, test_accuracy_list 

In [ ]:
import os

def train_model_bis(model, trainloader, testloader, epochs, is_scheduler, is_mixup, save_dir="output"):
    os.makedirs(save_dir, exist_ok=True)  # Ensure save directory exists
    log_path = os.path.join(save_dir, "training_logs.txt")
    
    train_accuracy_list, test_accuracy_list = [], []
    train_loss_list, test_loss_list = [], []

    with open(log_path, "w") as f:  # Open log file in write mode
        for epoch in range(epochs):
            model.train()
            running_loss = 0.0
            correct_train = 0
            total_train = 0

            for images, labels in trainloader:
                images, labels = images.to(device), labels.to(device)

                if is_mixup:
                    images, labels = mixup_transform(images, labels)
                    
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

                # Training accuracy
                _, predicted = outputs.max(1)

                if is_mixup:
                    labels = labels.argmax(dim=1)
                    
                correct_train += predicted.eq(labels).sum().item()
                total_train += labels.size(0)

            train_accuracy = 100 * correct_train / total_train
            train_accuracy_list.append(train_accuracy)
            train_loss_list.append(running_loss / len(trainloader))

            # Evaluate on test set
            model.eval()
            correct_test = 0
            total_test = 0
            test_loss = 0.0

            with torch.no_grad():
                for images, labels in testloader:
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    test_loss += loss.item()

                    _, predicted = outputs.max(1)
                    correct_test += predicted.eq(labels).sum().item()
                    total_test += labels.size(0)

            test_accuracy = 100 * (correct_test / total_test)
            test_accuracy_list.append(test_accuracy)
            test_loss_list.append(test_loss / len(testloader))

            # If using a scheduler
            current_lr = optimizer.param_groups[0]['lr']
            if is_scheduler:
                scheduler.step(test_loss / len(testloader))
                if current_lr != optimizer.param_groups[0]['lr']:
                    print(f"LEARNING RATE UPDATE TO:{optimizer.param_groups[0]['lr']}")
                    f.write(f"LEARNING RATE UPDATE TO:{optimizer.param_groups[0]['lr']}\n")

            # Logging epoch results
            epoch_log = (f"Epoch {epoch+1}: Train Loss: {running_loss/len(trainloader):.4f}, "
                         f"Train Acc: {train_accuracy:.2f}%, Test Acc: {test_accuracy:.2f}% "
                         f"Learning Rate: {optimizer.param_groups[0]['lr']}\n")

            print(epoch_log.strip())  # Print in the notebook
            f.write(epoch_log)  # Save to log file

    print("Training complete my boss")
    
    ####### PLOT THE LOSS AND ACCURACY PER EPOCH AND SAVE ########
    epochs_range = range(1, epochs+1)
    fig, axes = plt.subplots(1, 2, figsize=(12, 5)) 

    # Loss plot
    axes[0].plot(epochs_range, train_loss_list, label="Training loss", color="blue")
    axes[0].plot(epochs_range, test_loss_list, label="Testing loss", color="red")
    axes[0].set_title("Training and Testing Loss")
    axes[0].set_xlabel("Epochs")
    axes[0].set_ylabel("Loss")
    axes[0].legend()

    # Accuracy plot
    axes[1].plot(epochs_range, train_accuracy_list, label="Training accuracy", color="blue")
    axes[1].plot(epochs_range, test_accuracy_list, label="Testing accuracy", color="red")
    axes[1].set_title("Training and Testing Accuracy")
    axes[1].set_xlabel("Epochs")
    axes[1].set_ylabel("Accuracy (%)")
    axes[1].legend()

    plt.tight_layout()

    # Save the plot
    plot_path = os.path.join(save_dir, "training_plot.png")
    plt.savefig(plot_path)
    plt.show()

    print(f"Training logs saved to: {log_path}")
    print(f"Training plot saved to: {plot_path}")


In [ ]:
# Train the model
#train_model_bis(model, trainloader, testloader, epochs=num_epochs, is_scheduler=is_scheduler, is_mixup=is_mixup, save_dir="output")
train_model(model, trainloader, testloader, epochs=num_epochs, is_scheduler=is_scheduler)

In [ ]:
# # Save the model
# check_point = {
#     "epoch": num_epochs,
#     "model_state": model.state_dict(),
#     "optimizer": "Adam",
#     "optim_state": optimizer.state_dict() ,
#     "is_scheduler": is_scheduler
# }
# if check_point["is_scheduler"]:
#     check_point["scheduler"]=scheduler
    
# torch.save(check_point, './model_state/resnet18_model_simple.pth')

In [ ]:
# # We load the dictionary
# loaded_cpt = torch.load('./model_state/resnet18_model_simple.pth')

# new_model = ResNet18()
# new_model.load_state_dict(loaded_cpt['model_state'])

In [ ]:
# TO DO:

# 1) Presentation of the assigned paper (ResNet)

# 2) Hyperparameter exploration strategy (C10 full)

#   a) Architecture HP
#       - Type (ResNet, DenseNet, etc.)
#       - Depth (nb of layers)
#       - Width (nb of filter per layer, growthrate, etc.)

#   b) Training
#       - Optimizer (SGD, Adam, etc.)
#       - Learning rate, scheduler
#       - Nb of epochs
#       - Regularization ( weight decay)
#       - Data augmentation (to struggle overfitting)
#       - batch size


In [ ]:
# Currently, we are underfitting, we have to overfit first then fight overfitting.
# I've increased the batch size and as a result, the accuracy and loss increased and decreased respectively. 
# However, the model is still not overfitting, why? -> No generalization power? Model capacity limitation? -> more layer? change the achitecture?
# --> use the full dataset to see if the problem is persisting 

# Do we have to remove the transform (DA) ?

In [ ]:
# print head dataset
